# Model Training

In [1]:
from torchvision import transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
from PIL import Image # import PIL library

In [2]:
# Define a custom dataset class that inherits from torch.utils.data.Dataset
class CustomDataset(Dataset):
    def __init__(self, root):
        # Root is the directory that contains all the class folders
        self.root = root
        self.folders_paths = [os.path.join(root, folder) for folder in os.listdir(root)]
        self.classes = [folder.split("/")[-1] for folder in self.folders_paths]
        self.files = []
        for label, folder_path in enumerate(self.folders_paths):
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                self.files.append((file_path, label))

    def __getitem__(self, idx):
        # Return a sample and its corresponding label
        file_path, label = self.files[idx]
        # Open the image using PIL
        image = Image.open(file_path).convert('RGB')
        # Note: This returns an image in the form of a PIL object
        return image, label

    def __len__(self):
        # Return the number of samples
        return len(self.files)

    
# Create an instance of the custom dataset with a given root directory
root_dir = "final_data"
data = CustomDataset(root_dir)

# Split the dataset into train (60%), test (20%) and validation (20%) sets
train_size = int(0.6 * len(data))
test_size = int(0.2 * len(data))
val_size = len(data) - train_size - test_size
train_data, test_data, val_data = random_split(data, [train_size, test_size, val_size])

# Define different transformations for each dataset
train_transform  =   transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1),
        transforms.RandomAffine(degrees=40, translate=None, scale=(1, 2), shear=15),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Create a wrapper class to apply the transformations on the fly
class TransformDataset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

# Create transformed datasets using the wrapper class
train_data = TransformDataset(train_data, transform=train_transform)
test_data = TransformDataset(test_data, transform=transform_test)
val_data = TransformDataset(val_data, transform=transform_test)

# Create data loaders for each dataset with a given batch size
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)



In [3]:
# TODO: Build and train your network
from torch import nn
import torch.optim as optim
import torch
import numpy as np
from tqdm import tqdm
device = "cuda"

class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input tensor
        out = self.linear(x)
        return out

# Example usage
# instantiate the model
input_size = 224 * 224 * 3
num_classes = 8
net = LogisticRegression(input_size, num_classes).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimiser = optim.SGD(net.parameters(), lr=0.05)

In [4]:
def validation(net,dl_test):
    net.eval()
    correct=0
    total = 0
    for i, data in enumerate(dl_test, 0):
        # Get data from each batch. Data is in the format of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # Reset the parameter gradients
        optimiser.zero_grad()

        # Forward pass, backward pass, and optimise
        outputs = net(inputs)
        
        correct += (torch.argmax(outputs,-1) == labels).float().sum()
        total += len(labels)

    return correct/total

In [5]:
import copy
from tqdm import tqdm 


max_accuracy = 0
for epoch in range(200):  # Train for 200 epochs
    net.train()
    running_loss = 0.0
    train_acc = 0.0
    correct = 0
    pbar = tqdm(train_loader)

    for i, data in enumerate(pbar, 0):
        # Get data from each batch. Data is in the format of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # Reset the parameter gradients
        optimiser.zero_grad()

        # Forward pass, backward pass, and optimise
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()

        # Print 
        pbar.set_description(f"loss: {str(loss.item())}")

        running_loss += loss.item()
        correct += (torch.argmax(outputs,-1) == labels).float().sum()
    
    train_acc = correct/len(train_data)
    validation_acc = validation(net,val_loader)

    print(f'[Epoch: {epoch + 1}] train_loss: {running_loss / 127:.3f}, train_acc: {train_acc}, val_acc: {validation_acc}')

    if  validation_acc > max_accuracy:
        max_accuracy = validation_acc
        best_model = copy.deepcopy(net)
print('Finished Training')

loss: 693.3958129882812:  12%|█▏        | 15/127 [00:02<00:16,  6.64it/s] c:\Users\Raven\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
loss: 525.00244140625: 100%|██████████| 127/127 [00:19<00:00,  6.36it/s]   


[Epoch: 1] train_loss: 630.218, train_acc: 0.15225425362586975, val_acc: 0.17073170840740204


loss: 439.0318603515625: 100%|██████████| 127/127 [00:19<00:00,  6.50it/s] 


[Epoch: 2] train_loss: 631.825, train_acc: 0.1697462499141693, val_acc: 0.1648189127445221


loss: 564.4920654296875: 100%|██████████| 127/127 [00:19<00:00,  6.43it/s] 


...

[Epoch: 198] train_loss: 595.674, train_acc: 0.17713722586631775, val_acc: 0.16555801033973694


loss: 602.2128295898438: 100%|██████████| 127/127 [00:19<00:00,  6.40it/s] 


[Epoch: 199] train_loss: 604.882, train_acc: 0.1803399920463562, val_acc: 0.16925351321697235


loss: 740.0032348632812: 100%|██████████| 127/127 [00:19<00:00,  6.44it/s] 


[Epoch: 200] train_loss: 581.223, train_acc: 0.17467357218265533, val_acc: 0.15890613198280334
Finished Training


## Save/Load model.

In [6]:
model_dir = os.path.join(os.getcwd(), 'baseline_model', 'logistic_regression')
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
torch.save(best_model, os.path.join(model_dir, 'model.pth'))
best_model = torch.load(os.path.join(model_dir, 'model.pth'))

## Validate the models.

In [7]:
print(f"Accuracy on validation data: {validation(best_model, test_loader)}")

Accuracy on validation data: 0.210643008351326


In [8]:
transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

def validation2(net,image_path,transform):
    net.eval()
    correct=0
    total = 0
    image = Image.open(image_path).convert('RGB')
    inputs = transform(image)
    inputs = inputs.cuda()
    inputs = torch.unsqueeze(inputs, 0)
      
        # forward + backward + optimize
    outputs = net(inputs)

    return torch.argmax(outputs,-1)

validation2(best_model, 'IMG_0640.PNG', transform_test)

tensor([0], device='cuda:0')